In [78]:
!ls Dataset/arxiv/

abstract_noun_clean.pickle	  metadata
abstract_noun_clean_strat.pickle  metadata_arxiv.tar.gz
abstract_noun.pickle		  tar_members.pickle
arxiv_abstract.csv		  test_all.csv
arxiv_author.csv		  test_mul.csv
arxiv_categories.csv		  tfidf_w_embed_vector.pickle
arxiv_data.json			  tf_word_vector.csv
arxiv_main.csv			  train_2016.csv
arxiv_main.gsheet		  train_2017.csv
cc.en.300.bin			  train_all.csv
cc.en.300.vec			  trained_first_model.pickle
main_test.csv			  train_mul.csv
main_train.csv


In [0]:
import pandas as pd

In [0]:
categories = pd.read_csv("Dataset/arxiv/arxiv_categories.csv",names=["id","cat","sub"])
main = pd.read_csv("Dataset/arxiv/arxiv_main.csv",names=["id","create_date","update_date"])

In [81]:
categories[categories["id"]=="1601.00124"]

,id,cat,sub
1036529,1601.00124,math,math.CA


In [0]:
from datetime import datetime
year_month = main.create_date.apply(lambda x:(datetime.strptime(x,"%Y-%m-%d").year,datetime.strptime(x,"%Y-%m-%d").month))

In [0]:
pd_main = main.join(pd.DataFrame(list(year_month.values),columns=["year","month"]))

In [0]:
pd_cat_2016 = pd_main[pd_main.year==2016].merge(categories,left_on="id",right_on="id")

In [0]:
pd_cat_2017 = pd_main[pd_main.year==2017].merge(categories,left_on="id",right_on="id")

In [14]:
pd_cat_2016.drop_duplicates("id",keep="first").shape

(113436, 7)

use 2016 dataset for training

In [0]:
pd_test = pd.read_csv("Dataset/arxiv/main_test.csv",dtype="str")
pd_test  = pd_test.set_index("id")
pd_train = pd.read_csv("Dataset/arxiv/main_train.csv",dtype="str")
pd_train = pd_train.set_index("id")

In [0]:
pd_cat = pd.read_csv("Dataset/arxiv/arxiv_categories.csv",names=["id","cat","sub_cat"],index_col="id")

In [0]:
pd_abstract = pd.read_csv("Dataset/arxiv/arxiv_abstract.csv",names=["id","title","abstract"],index_col="id")

In [0]:
pd_main = pd_cat_2016.set_index("id")

In [0]:
pd_2016_abs = pd_main.join(pd_abstract)

In [0]:
pd_train = pd_cat_2016.set_index("id")

In [15]:
pd_train.index.unique()

Index(['1609.06953', '1605.08863', '1603.03620', '1604.02220', '1603.00933',
       '1611.03500', '1605.07922', '1607.07455', '1612.04911', '1605.09032',
       ...
       '1606.06113', '1609.07749', '1601.00322', '1608.04951', '1608.01221',
       '1606.06677', '1605.04886', '1603.04997', '1609.03645', '1601.04716'],
      dtype='object', name='id', length=113436)

In [16]:
pd_train.reset_index().drop_duplicates("id",keep="first").groupby("cat").count()

,id,create_date,update_date,year,month,sub
cat,,,,,,
astro-ph,13215,13215,4465,13215,13215,13215
cond-mat,14624,14624,5776,14624,14624,14624
cs,20770,20770,7471,20770,20770,20770
gr-qc,2607,2607,1474,2607,2607,2607
hep-ex,780,780,456,780,780,780
hep-lat,543,543,226,543,543,543
hep-ph,4751,4751,2714,4751,4751,4751
hep-th,3614,3614,2320,3614,3614,3614
math,28503,28503,12814,28503,28503,28503


In [87]:
pd_2016_abs.loc["1601.00057"]

,create_date,update_date,year,month,cat,sub,title,abstract
id,,,,,,,,
1601.00057,2016-01-01,2016-12-11,2016,1,hep-th,hep-th,Tracking Dark Energy from Axion-Gauge Field Co...,We propose a toy model of Dark Energy in which...
1601.00057,2016-01-01,2016-12-11,2016,1,astro-ph,astro-ph.CO,Tracking Dark Energy from Axion-Gauge Field Co...,We propose a toy model of Dark Energy in which...
1601.00057,2016-01-01,2016-12-11,2016,1,gr-qc,gr-qc,Tracking Dark Energy from Axion-Gauge Field Co...,We propose a toy model of Dark Energy in which...
1601.00057,2016-01-01,2016-12-11,2016,1,hep-ph,hep-ph,Tracking Dark Energy from Axion-Gauge Field Co...,We propose a toy model of Dark Energy in which...


In [88]:
import csv
#test_list = []
#test_not_found = []
file = open("Dataset/arxiv/train_2016.csv","w")
main_writer = csv.writer(file)
#for i,x in enumerate(pd_train.index.unique()):
id_before = ""
for i,x in enumerate(pd_2016_abs.reset_index().values):
  if i%10000 == 0:
    print(i,end=" ")
  #print(x)
  #try:
    #pd_temp_cat = pd_cat[pd_cat["id"]==str(x[0])]
  #print(x)
  pd_id = x[0]
  pd_date = x[1]
  if id_before!=pd_id:
    if id_before!="":
      main_writer.writerow([old_x[0],old_x[1],old_x[2],old_x[3],old_x[4],list_cat[0],"|".join(list_cat),"|".join(list_sub_cat),old_x[7],old_x[8]])
    id_before = pd_id
    list_cat = []
    list_sub_cat = []    
  if x[5] not in list_cat:
    list_cat.append(x[5])
  if x[6] not in list_sub_cat:
    list_sub_cat.append(x[6])
  old_x = x
  #if i > 20:
  #  break    
main_writer.writerow([x[0],x[1],x[2],x[3],x[4],list_cat[0],"|".join(list_cat),"|".join(list_sub_cat),x[7],x[8]])
file.close()

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 

In [0]:
pd_cat_2017_abs = pd_cat_2017.set_index("id")
pd_cat_2017_abs = pd_cat_2017_abs.join(pd_abstract)

In [89]:
import csv
#test_list = []
#test_not_found = []
file = open("Dataset/arxiv/train_2017.csv","w")
main_writer = csv.writer(file)
#for i,x in enumerate(pd_train.index.unique()):
id_before = ""
for i,x in enumerate(pd_cat_2017_abs.reset_index().values):
  if i%10000 == 0:
    print(i,end=" ")
  #print(x)
  #try:
    #pd_temp_cat = pd_cat[pd_cat["id"]==str(x[0])]
  #print(x)
  pd_id = x[0]
  pd_date = x[1]
  if id_before!=pd_id:
    if id_before!="":
      main_writer.writerow([old_x[0],old_x[1],old_x[2],old_x[3],old_x[4],list_cat[0],"|".join(list_cat),"|".join(list_sub_cat),old_x[7],old_x[8]])
    id_before = pd_id
    list_cat = []
    list_sub_cat = []    
  if x[5] not in list_cat:
    list_cat.append(x[5])
  if x[6] not in list_sub_cat:
    list_sub_cat.append(x[6])
  old_x = x    
  #if i > 20:
  #  break    
main_writer.writerow([x[0],x[1],x[2],x[3],x[4],list_cat[0],"|".join(list_cat),"|".join(list_sub_cat),x[7],x[8]])
file.close()

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 

In [0]:
pd_read_2016 = pd.read_csv("Dataset/arxiv/train_2016.csv",names = ["arxiv_id","create_date","update_date","year","month","cat","cats","sub_cats","title","abstract"],dtype="str")

In [91]:
pd_read_2016[pd_read_2016.arxiv_id=="1601.00057"]

,arxiv_id,create_date,update_date,year,month,cat,cats,sub_cats,title,abstract
5,1601.00057,2016-01-01,2016-12-11,2016,1,hep-th,hep-th|astro-ph|gr-qc|hep-ph,hep-th|astro-ph.CO|gr-qc|hep-ph,Tracking Dark Energy from Axion-Gauge Field Co...,We propose a toy model of Dark Energy in which...


In [92]:
pd_read_2016.groupby("cat").count()

,arxiv_id,create_date,update_date,year,month,cats,sub_cats,title,abstract
cat,,,,,,,,,
astro-ph,13215,13215,4465,13215,13215,13215,13215,13215,13215
cond-mat,14624,14624,5776,14624,14624,14624,14624,14624,14624
cs,20770,20770,7471,20770,20770,20770,20770,20770,20770
gr-qc,2607,2607,1474,2607,2607,2607,2607,2607,2607
hep-ex,780,780,456,780,780,780,780,780,780
hep-lat,543,543,226,543,543,543,543,543,543
hep-ph,4751,4751,2714,4751,4751,4751,4751,4751,4751
hep-th,3614,3614,2320,3614,3614,3614,3614,3614,3614
math,28503,28503,12814,28503,28503,28503,28503,28503,28503


In [0]:
pd_read_2017 = pd.read_csv("Dataset/arxiv/train_2017.csv",names = ["arxiv_id","create_date","update_date","year","month","cat","cats","sub_cats","title","abstract"],dtype="str")

In [94]:
pd_read_2017.cat.unique()

array(['math', 'cs', 'physics', 'hep-th', 'math-ph', 'nlin', 'cond-mat',
       'quant-ph', 'hep-ph', 'astro-ph', 'gr-qc', 'nucl-th', 'hep-lat',
       'stat', 'q-bio', 'hep-ex', 'q-fin', 'nucl-ex', 'eess', 'econ'],
      dtype=object)

In [95]:
pd_read_2017.dtypes

arxiv_id       object
create_date    object
update_date    object
year           object
month          object
cat            object
cats           object
sub_cats       object
title          object
abstract       object
dtype: object

In [96]:
pd_read_2017.groupby("cat").count().reset_index()

,cat,arxiv_id,create_date,update_date,year,month,cats,sub_cats,title,abstract
0,astro-ph,13269,13269,4322,13269,13269,13269,13269,13269,13269
1,cond-mat,14416,14416,5467,14416,14416,14416,14416,14416,14416
2,cs,27085,27085,9130,27085,27085,27085,27085,27085,27085
3,econ,59,59,22,59,59,59,59,59,59
4,eess,430,430,114,430,430,430,430,430,430
5,gr-qc,2662,2662,1438,2662,2662,2662,2662,2662,2662
6,hep-ex,879,879,454,879,879,879,879,879,879
7,hep-lat,650,650,220,650,650,650,650,650,650
8,hep-ph,4516,4516,2491,4516,4516,4516,4516,4516,4516
9,hep-th,3551,3551,2193,3551,3551,3551,3551,3551,3551


In [97]:
pd_train_2017 = pd.DataFrame()
pd_test_2017 = pd.DataFrame()
for x in  pd_read_2017.groupby("cat").count().reset_index().values:
  cat = x[0]
  total = int(x[1] * 0.3)
  pd_test_2017 = pd_test_2017.append(pd_read_2017[pd_read_2017.cat==cat].iloc[:total])
  pd_train_2017 = pd_train_2017.append(pd_read_2017[pd_read_2017.cat==cat].iloc[total:])
  print(pd_test_2017.shape)
  print(pd_train_2017.shape)
  #break

(3980, 10)
(9289, 10)
(8304, 10)
(19381, 10)
(16429, 10)
(38341, 10)
(16446, 10)
(38383, 10)
(16575, 10)
(38684, 10)
(17373, 10)
(40548, 10)
(17636, 10)
(41164, 10)
(17831, 10)
(41619, 10)
(19185, 10)
(44781, 10)
(20250, 10)
(47267, 10)
(29304, 10)
(68396, 10)
(29746, 10)
(69428, 10)
(30007, 10)
(70038, 10)
(30139, 10)
(70346, 10)
(30518, 10)
(71233, 10)
(33705, 10)
(78672, 10)
(34209, 10)
(79849, 10)
(34423, 10)
(80350, 10)
(35921, 10)
(83846, 10)
(37125, 10)
(86656, 10)


In [0]:
pd_train_all = pd_read_2016.append(pd_train_2017)


In [99]:
pd_train_all.groupby("cat").count()

,arxiv_id,create_date,update_date,year,month,cats,sub_cats,title,abstract
cat,,,,,,,,,
astro-ph,22504,22504,7476,22504,22504,22504,22504,22504,22504
cond-mat,24716,24716,9585,24716,24716,24716,24716,24716,24716
cs,39730,39730,13608,39730,39730,39730,39730,39730,39730
econ,42,42,15,42,42,42,42,42,42
eess,301,301,74,301,301,301,301,301,301
gr-qc,4471,4471,2460,4471,4471,4471,4471,4471,4471
hep-ex,1396,1396,764,1396,1396,1396,1396,1396,1396
hep-lat,998,998,368,998,998,998,998,998,998
hep-ph,7913,7913,4407,7913,7913,7913,7913,7913,7913


In [0]:
pd_train_all.to_csv("Dataset/arxiv/train_all.csv",index=False)

In [0]:
pd_test_2017.to_csv("Dataset/arxiv/test_all.csv",index=False)

In [102]:
pd_train_all.shape

(200092, 10)

In [103]:
pd_test_2017.shape

(37125, 10)